### 第一步：从"市场"导入预置的数据集
目前ModelArts SDK/API暂时不支持对"市场"的操作，在本地使用OBS SDK上传数据；或使用web console导入数据
(ModelArts notebook中无法访问网络，只能通过web console操作导入数据)

In [2]:
## 准备SDK
!pip install esdk-obs-python

Looking in indexes: https://pypi.mirrors.ustc.edu.cn/simple


In [ ]:
%env USER_ACCESS_KEY＝my_ak
%env USER_SECRET_ACCESS_KEY=my_sk

In [3]:
import os
from obs import ObsClient

AK=os.getenv('USER_ACCESS_KEY')
SK=os.getenv('USER_SECRET_ACCESS_KEY')
    
obs_endpoint=os.getenv('MINER_OBS_URL')
if obs_endpoint is None:
    obs_endpoint='obs.cn-north-1.myhuaweicloud.com'


obsClient = ObsClient(
    access_key_id=AK,
    secret_access_key=SK,
    server=obs_endpoint
)

In [4]:
TARGET_BUCKET = 'ma-workflow'
resp = obsClient.headBucket(TARGET_BUCKET)
if resp.status == 404 and obsClient.createBucket(TARGET_BUCKET).status >= 300:
    print("Failed to create bucket{}".format(TARGET_BUCKET))


In [5]:
resp = obsClient.putFile(TARGET_BUCKET, 'mnist/code/train_mnist.py', file_path='./train_mnist.py')
if resp.status >= 300:
    print('Failed to put file')

In [6]:
print(resp.body)

{'versionId': 'G0011168E11F9096FFFF8052102F661D', 'etag': '"224153ce0841c008e84d34819b6df347"', 'objectUrl': 'https://ma-workflow.obs.cn-north-1.myhuaweicloud.com/mnist/code/train_mnist.py'}


In [8]:
from modelarts import config
from modelarts.client.api import *

创建ModelArts客户端，注意需要在HOME目录下配置.modelarts/config.json
详情参见[初始化ModelArts SDK](https://support.huaweicloud.com/sdkreference-modelarts/modelarts_04_0006.html)

In [10]:
maClient = config.create_client(context='default')

FileNotFoundError: [Errno 2] No such file or directory: '/home/lance/.modelarts/config.json'